In [11]:
import torch
import rasterio

# Define the directory path where the rasters are located
directory = '/nfs/home/innocenti/igarss-fmars/data/ithaca-val/gt_45'
# Get a list of all raster files in the directory
raster_files = rasterio.os.listdir(directory)
# sort the list
raster_files.sort()
# Create an empty list to store the loaded rasters
labels = []
# Iterate over each raster file
for file in raster_files:
    # Open the raster file using rasterio
    with rasterio.open(directory + '/' + file) as src:
        # Read the raster data
        raster_data = src.read(1)  # Change the band number if needed
        # shift all labels by 1, set 255 to 1
        raster_data = raster_data + 1
        raster_data[raster_data == 256] = 0
        # Convert the raster data to a torch tensor
        labels.append(torch.from_numpy(raster_data))
        

In [12]:
print(len(labels))
print(labels[0].shape)
print(labels[0].unique())

45
torch.Size([512, 512])
tensor([0, 1, 2, 3], dtype=torch.uint8)


In [13]:
import torch
import rasterio

# Define the directory path where the rasters are located
directory = '/nfs/home/innocenti/igarss-fmars/output_images/mic_sampler_val_075'
# Get a list of all raster files in the directory
raster_files = rasterio.os.listdir(directory)
# sort the list
raster_files.sort()
# Create an empty list to store the loaded rasters
inferences = []
# Iterate over each raster file
for file in raster_files:
    # Open the raster file using rasterio
    with rasterio.open(directory + '/' + file) as src:
        # Read the raster data
        raster_data = src.read(1)  # Change the band number if needed
        
        # Convert the raster data to a torch tensor
        inferences.append(torch.from_numpy(raster_data))

In [14]:
print(len(inferences))
print(inferences[0].shape)
print(inferences[0].unique())

45
torch.Size([512, 512])
tensor([0, 1, 2, 3], dtype=torch.uint8)


In [15]:
from torchmetrics import Accuracy, JaccardIndex

# Convert the labels and inferences to torch tensors
labels_tensor = torch.stack(labels)
inferences_tensor = torch.stack(inferences)

# Create the accuracy metric
accuracy_perclass = Accuracy(num_classes=4, task='multiclass', average='none')
accuracy_micro = Accuracy(num_classes=4, task='multiclass', average='micro')
accuracy_macro = Accuracy(num_classes=4, task='multiclass', average='macro')
accuracy_ignorebg = Accuracy(num_classes=4, task='multiclass', average='micro', ignore_index=0)

# Compute the accuracy between labels and inferences
accuracy_perclass_value = accuracy_perclass(inferences_tensor, labels_tensor)
accuracy_micro_value = accuracy_micro(inferences_tensor, labels_tensor)
accuracy_macro_value = accuracy_macro(inferences_tensor, labels_tensor)
accuracy_ignorebg_value = accuracy_ignorebg(inferences_tensor, labels_tensor)

# Create the IoU metric
iou_perclass = JaccardIndex(num_classes=4, task='multiclass', average='none')
iou_micro = JaccardIndex(num_classes=4, task='multiclass', average='micro')
iou_macro = JaccardIndex(num_classes=4, task='multiclass', average='macro') 
iou_ignorebg = JaccardIndex(num_classes=4, task='multiclass', average='micro', ignore_index=0)

# Compute the mIoU between labels and inferences
iou_perclass_value = iou_perclass(inferences_tensor, labels_tensor)
iou_micro_value = iou_micro(inferences_tensor, labels_tensor)
iou_macro_value = iou_macro(inferences_tensor, labels_tensor)
iou_ignorebg_value = iou_ignorebg(inferences_tensor, labels_tensor)

# Print the accuracy and mIoU values
print('Accuracy per class:', accuracy_perclass_value)
print('Accuracy micro:', accuracy_micro_value)
print('Accuracy macro:', accuracy_macro_value)
print('Accuracy ignore bg:', accuracy_ignorebg_value)
print('IoU per class:', iou_perclass_value)
print('IoU micro:', iou_micro_value)
print('IoU macro:', iou_macro_value)
print('IoU ignore bg:', iou_ignorebg_value)

Accuracy per class: tensor([0.4835, 0.5895, 0.7198, 0.8187])
Accuracy micro: tensor(0.6662)
Accuracy macro: tensor(0.6529)
Accuracy ignore bg: tensor(0.7328)
IoU per class: tensor([0.3000, 0.4648, 0.6266, 0.6385])
IoU micro: tensor(0.4995)
IoU macro: tensor(0.5075)
IoU ignore bg: tensor(0.7016)
